# tutorial https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

## Note to Jose
Working version but not fully understood yet
The next commented think can allow to remove stuff

In [1]:
# Download TorchVision repo to use some files from
# references/detection
!git clone git@github.com:jcmayoral/PyTorch-YOLOv3.git yolov3

fatal: destination path 'yolov3' already exists and is not an empty directory.


In [2]:
from yolov3.models import Darknet

#model_4classes = Darknet(config_path="yolov3/config/yolov3-custom.cfg")

In [3]:
#nparams = sum(p.numel() for p in model_4classes.parameters())
#print (f"{nparams:,}")

In [4]:
!ls
!pwd

'attempt yolo.ipynb'		   __pycache__
 checkpoints			   rcnntutorial.ipynb
 coco_eval.py			   scalar
 coco_utils.py			   svmpersons.ipynb
 decisiontreepersons.ipynb	   testingyolo
 engine.py			   transforms.py
 linearregressionpersons.ipynb	   utils.py
 logisticregressionpersons.ipynb   vision
 output				   yolov3
 pointnet.ipynb
/home/jose/ros_ws/src/gr_perception/gr_ml/nb


In [5]:
from __future__ import division

from yolov3.models import *
from yolov3.utils.logger import *
from yolov3.utils.utils import *
from yolov3.utils.datasets import *
from yolov3.utils.parse_config import *
from yolov3.test import evaluate

from terminaltables import AsciiTable

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim


"""
parser.add_argument("--epochs", type=int, default=100, help="number of epochs")
parser.add_argument("--batch_size", type=int, default=8, help="size of each image batch")
parser.add_argument("--gradient_accumulations", type=int, default=2, help="number of gradient accums before step")
parser.add_argument("--model_def", type=str, default="config/yolov3.cfg", help="path to model definition file")
parser.add_argument("--data_config", type=str, default="config/coco.data", help="path to data config file")
parser.add_argument("--pretrained_weights", type=str, help="if specified starts from checkpoint model")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
parser.add_argument("--evaluation_interval", type=int, default=1, help="interval evaluations on validation set")
parser.add_argument("--compute_map", default=False, help="if True computes mAP every tenth batch")
parser.add_argument("--multiscale_training", default=True, help="allow for multi-scale training")
opt = parser.parse_args()
print(opt)
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

# Get data configuration
data_config = parse_data_config("yolov3/config/jose.data")
#data_config = { line.split()[0] : line.split()[1] for line in open("yolov3/config/jose.data") }

#parse_data_config(opt.data_config)bb
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

print(class_names)

# Initiate model
model = Darknet("yolov3/config/yolov3-custom.cfg").to(device)
model.apply(weights_init_normal)

# If specified we start from checkpoint
"""
if opt.pretrained_weights:
    if opt.pretrained_weights.endswith(".pth"):
        model.load_state_dict(torch.load(opt.pretrained_weights))
    else:
        model.load_darknet_weights(opt.pretrained_weights)
"""



/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWa

['Lethal', 'Danger', 'Warning', 'Safe']


'\nif opt.pretrained_weights:\n    if opt.pretrained_weights.endswith(".pth"):\n        model.load_state_dict(torch.load(opt.pretrained_weights))\n    else:\n        model.load_darknet_weights(opt.pretrained_weights)\n'

In [6]:
bs = 5
nepochs = 10
ncpu = 4
gradient_accumulations = 1
evaluation_interval = 2
checkpoint_interval = 2
img_size = 416

In [7]:
!pwd


/home/jose/ros_ws/src/gr_perception/gr_ml/nb


In [ ]:
# Get dataloader
#For some reason augment or multiscale fails
dataset = ListDataset(train_path, augment=False, multiscale=False)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=bs,
    shuffle=False,
    num_workers=ncpu,
    pin_memory=False,
    collate_fn=dataset.collate_fn,
)

optimizer = torch.optim.Adam(model.parameters())

metrics = [
    "grid_size",
    "loss",
    "x",
    "y",
    "w",
    "h",
    "conf",
    "cls",
    "cls_acc",
    "recall50",
    "recall75",
    "precision",
    "conf_obj",
    "conf_noobj",
]

try:
    os.makedirs("testingyolo")
except:
    pass

#logger = Logger("testingyolo/logs")


for epoch in range(nepochs):
    model.train()
    start_time = time.time()
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        print(_)
        batches_done = len(dataloader) * epoch + batch_i

        imgs = Variable(imgs.to(device))
        targets = Variable(targets.to(device), requires_grad=False)

        loss, outputs = model(imgs, targets)
        loss.backward()

        if batches_done % gradient_accumulations:
            # Accumulates gradient before each step
            optimizer.step()
            optimizer.zero_grad()

        # ----------------
        #   Log progress
        # ----------------

        log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, nepochs, batch_i, len(dataloader))

        metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

        # Log metrics at each YOLO layer
        for i, metric in enumerate(metrics):
            formats = {m: "%.6f" for m in metrics}
            formats["grid_size"] = "%2d"
            formats["cls_acc"] = "%.2f%%"
            row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
            metric_table += [[metric, *row_metrics]]

            # Tensorboard logging
            tensorboard_log = []
            for j, yolo in enumerate(model.yolo_layers):
                for name, metric in yolo.metrics.items():
                    if name != "grid_size":
                        tensorboard_log += [(f"{name}_{j+1}", metric)]
            tensorboard_log += [("loss", loss.item())]
            #logger.list_of_scalars_summary(tensorboard_log, batches_done)

        log_str += AsciiTable(metric_table).table
        log_str += f"\nTotal loss {loss.item()}"

        # Determine approximate time left for epoch
        epoch_batches_left = len(dataloader) - (batch_i + 1)
        time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
        log_str += f"\n---- ETA {time_left}"

        print(log_str)

        model.seen += imgs.size(0)

    if epoch % evaluation_interval == 0:
        print("\n---- Evaluating Model ----")
        # Evaluate the model on the validation set
        precision, recall, AP, f1, ap_class = evaluate(
            model,
            path=valid_path,
            iou_thres=0.5,
            conf_thres=0.5,
            nms_thres=0.5,
            img_size=img_size,
            batch_size=8,
        )
        """
        evaluation_metrics = [
            ("val_precision", precision.mean()),
            ("val_recall", recall.mean()),
            ("val_mAP", AP.mean()),
            ("val_f1", f1.mean()),
        ]
        logger.list_of_scalars_summary(evaluation_metrics, epoch)
        """
        # Print class APs and mAP
        ap_table = [["Index", "Class name", "AP"]]
        for i, c in enumerate(ap_class):
            ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
        print(AsciiTable(ap_table).table)
        print(f"---- mAP {AP.mean()}")

    if epoch % checkpoint_interval == 0:
        torch.save(model.state_dict(), f"testingyolo/checkpoints/yolov3_ckpt_%d.pth" % epoch)


index:5 of 137
index:0 of 137
index:10 of 137
index:15 of 137
index:11 of 137
index:6 of 137
index:1 of 137
index:16 of 137
index:12 of 137
index:17 of 137
index:2 of 137
index:7 of 137
index:13 of 137
index:18 of 137
index:3 of 137
index:8 of 137
index:14 of 137
index:19 of 137
colatein
[tensor([[0.0000, 2.0000, 0.3352, 0.4422, 0.1359, 0.3156],
        [0.0000, 2.0000, 0.9125, 0.4664, 0.1625, 0.3266]]), tensor([[1.0000, 2.0000, 0.3383, 0.4422, 0.1609, 0.3156],
        [1.0000, 1.0000, 0.8539, 0.5016, 0.2047, 0.3500]]), tensor([[2.0000, 1.0000, 0.7312, 0.4820, 0.2000, 0.3891],
        [2.0000, 2.0000, 0.3414, 0.4422, 0.1578, 0.3156]]), tensor([[3.0000, 2.0000, 0.3461, 0.4266, 0.1547, 0.3469]]), tensor([[4.0000, 1.0000, 0.3492, 0.4406, 0.1641, 0.3500]])]index:4 of 137
index:9 of 137
 HERE ('/media/WIP/yolov3dataset/openfield_all/images/image_12.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_13.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_14.jpg', '/media/WIP/yo

/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please 

index:45 of 137
index:46 of 137
index:47 of 137
index:48 of 137
index:49 of 137
colatein
[tensor([[0.0000, 2.0000, 0.7047, 0.4586, 0.1312, 0.2859]]), tensor([[1.0000, 2.0000, 0.7086, 0.4586, 0.1484, 0.2922]]), tensor([[2.0000, 2.0000, 0.7109, 0.4547, 0.1688, 0.2938]]), tensor([[3.0000, 2.0000, 0.7039, 0.4570, 0.1359, 0.2828]]), tensor([[4.0000, 2.0000, 0.7094, 0.4492, 0.1656, 0.2891]])] HERE ('/media/WIP/yolov3dataset/openfield_all/images/image_47.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_48.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_49.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_50.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_51.jpg')
colateout

---- [Epoch 0/10, Batch 0/28] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 13           | 26           | 52           |
| loss  

/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please 

index:50 of 137
index:51 of 137
index:52 of 137
index:53 of 137
index:54 of 137
colatein
[tensor([[0.0000, 2.0000, 0.7047, 0.4563, 0.1344, 0.2844]]), tensor([[1.0000, 2.0000, 0.7055, 0.4492, 0.1422, 0.2859]]), tensor([[2.0000, 2.0000, 0.7000, 0.4555, 0.1344, 0.2734]]), tensor([[3.0000, 2.0000, 0.7016, 0.4648, 0.1469, 0.3047]]), tensor([[4.0000, 2.0000, 0.7008, 0.4570, 0.1641, 0.2984]])] HERE ('/media/WIP/yolov3dataset/openfield_all/images/image_52.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_53.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_54.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_55.jpg', '/media/WIP/yolov3dataset/openfield_all/images/image_56.jpg')
colateout

---- [Epoch 0/10, Batch 1/28] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 13           | 26           | 52           |
| loss  

...


---- [Epoch 1/10, Batch 23/28] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 13           | 26           | 52           |
| loss       | 93.437027    | 61.879997    | 73.183121    |
| x          | 0.113048     | 0.084122     | 0.154291     |
| y          | 0.118634     | 0.167253     | 0.073882     |
| w          | 2.646430     | 0.796283     | 0.499892     |
| h          | 2.245610     | 1.035861     | 2.052930     |
| conf       | 87.546196    | 59.070217    | 69.555878    |
| cls        | 0.767102     | 0.726260     | 0.846250     |
| cls_acc    | 12.50%       | 37.50%       | 25.00%       |
| recall50   | 0.125000     | 0.000000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000539     | 0.000000     | 0.000000     |
| conf_obj   | 0.579630     | 0.384148     | 0.531129     |
| c

/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please 


---- [Epoch 1/10, Batch 24/28] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 13           | 26           | 52           |
| loss       | 91.136360    | 60.758419    | 72.090355    |
| x          | 0.136542     | 0.078428     | 0.093699     |
| y          | 0.155735     | 0.141928     | 0.072360     |
| w          | 1.897604     | 0.345269     | 0.263271     |
| h          | 0.468046     | 0.306430     | 1.203977     |
| conf       | 87.797211    | 59.147076    | 69.563202    |
| cls        | 0.681219     | 0.739289     | 0.893841     |
| cls_acc    | 20.00%       | 10.00%       | 0.00%        |
| recall50   | 0.000000     | 0.000000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000000     | 0.000000     | 0.000000     |
| conf_obj   | 0.555728     | 0.367107     | 0.528222     |
| c

/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/tmp/pip-req-build-4baxydiv/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please 


---- [Epoch 1/10, Batch 25/28] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 13           | 26           | 52           |
| loss       | 92.093475    | 60.038231    | 71.408112    |
| x          | 0.137216     | 0.061545     | 0.096147     |
| y          | 0.044792     | 0.060796     | 0.082250     |
| w          | 2.116665     | 0.148985     | 0.167462     |
| h          | 1.180634     | 0.122812     | 0.721854     |
| conf       | 87.906837    | 58.913658    | 69.569000    |
| cls        | 0.707327     | 0.730436     | 0.771397     |
| cls_acc    | 10.00%       | 10.00%       | 30.00%       |
| recall50   | 0.000000     | 0.000000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000000     | 0.000000     | 0.000000     |
| conf_obj   | 0.483829     | 0.425934     | 0.512865     |
| c

In [ ]:
a,b,c = next(iter(dataloader))

In [ ]:
a

In [ ]:
b.shape

In [ ]:
c

In [ ]:
labels=[]

In [ ]:
#list of pixels?
labels += c[:, 1].tolist()
# Rescale target
#targets[:, 2:] = xywh2xyxy(targets[:, 2:])
#targets[:, 2:] *= img_size


In [ ]:
print(imgs.shape)
arr = imgs.cpu().mul(255).permute(0,2,3,1).byte().numpy()
print(arr[0].shape, arr[0].dtype)

In [ ]:
Image.fromarray(arr[0])

In [ ]:
conf_thres = 0.001
nms_thres = 0.5

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(imgs)
    outputs = non_max_suppression(outputs, conf_thres=conf_thres, nms_thres=nms_thres)


In [ ]:
#(x1, y1, x2, y2, object_conf, class_score, class_pred)
outputs[0].shape

In [ ]:
boxes = outputs[0].cpu().numpy()[:,:4]
boxes.shape

In [ ]:
imgs.shape

In [ ]:
def get_rect(x, y, width, height):
    rect = [(x, y), (x+width, y), (x+width, y+height), (x, y+height), (x, y)]
    #rect = [(0, 0), (width, height)]
    return tuple(rect)

In [ ]:
from PIL import ImageDraw

image = Image.fromarray(arr[0])
print (image.size, image.mode)
pshape = imgs.shape

#overlay = Image.new("L", (pshape[3], pshape[2]), 0)
#draw = ImageDraw.Draw(overlay)
draw = ImageDraw.Draw(image)

rects = [get_rect(*p) for p in boxes]

for rect in rects:
    draw.line([tuple(p) for p in rect], width=1, fill=255)

    
#img = Image.merge("RGB", (image,overlay,image))
image

In [ ]:
# pick one image from the test set
img, img2 = dataset_test[2]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
print(img2.keys())
print(img2['boxes'])
mshape = img2['masks'].shape
arr = np.zeros((mshape[1],mshape[2]),dtype=np.uint8)
for i in range(mshape[0]):
    print ("load")
    arr += img2['masks'][i].mul(255).byte().numpy()
print (arr, np.unique(arr), arr.shape, type(arr))
image = Image.fromarray(arr)
image

In [ ]:
def get_rect(x, y, width, height):
    rect = [(x, y), (x+width, y), (x+width, y+height), (x, y+height), (x, y)]
    #rect = [(0, 0), (width, height)]
    
    return tuple(rect)

In [ ]:
from PIL import ImageDraw
pshape = prediction[0]['masks'].shape
boxes = prediction[0]['boxes'].cpu().numpy()
print(prediction[0]['masks'].shape)

arr = np.zeros((pshape[2],pshape[3]),dtype=np.uint8)
for i in range(pshape[0]):
    arr += prediction[0]['masks'][i,0].mul(255).byte().cpu().numpy()
image = Image.fromarray(arr)


overlay = Image.new("L", (pshape[3], pshape[2]), 0)
draw = ImageDraw.Draw(overlay)
rects = [get_rect(*p) for p in boxes]

for rect in rects:
    draw.line([tuple(p) for p in rect], width=3, fill=500)


print(image.size)
print(overlay.size)

img = Image.merge("RGB", (image,overlay,image))
(image,overlay)
img